In [33]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import numpy as np

In [35]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

### Lets test the image generator

In [45]:
img = load_img("data/train/nodog/4c324449-3cd3-43da-b407-d75b47c396b6.jpg")
x = img_to_array(img)
x = x.reshape((1,) + x.shape)
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='nodog', save_format='jpeg'):
    i += 1
    if i > 20:
        break 

### Create train and validation generators

In [73]:
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)


test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/train/',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        "data/test/",
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')



Found 286 images belonging to 2 classes.
Found 32 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


### Simple ConvNet

In [74]:
input_shape = (150, 150, 3)

In [75]:
input_img = Input(shape=input_shape)

conv_1 = Conv2D(32, (3,3), activation="relu")(input_img)
pool_1 = MaxPooling2D(pool_size=(2,2))(conv_1)

conv_2 = Conv2D(32, (3,3), activation="relu")(pool_1)
pool_2 = MaxPooling2D(pool_size=(2,2))(conv_2)

conv_3 = Conv2D(64, (3,3), activation="relu")(pool_2)
pool_3 = MaxPooling2D(pool_size=(2,2))(conv_3)

flattened = Flatten()(pool_3)
fc_1 = Dense(64, activation="relu")(flattened)
dropout_1 = Dropout(0.5)(fc_1) 
prediction = Dense(1, activation="sigmoid")(dropout_1)

model = Model(inputs=input_img, outputs=prediction)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [77]:
np.bincount(train_generator.classes)

array([143, 143])

In [78]:
model.fit_generator(
    train_generator,
    steps_per_epoch=500,
    epochs=2,
    validation_data=validation_generator,
    validation_steps=100
)

score = model.evaluate_generator(generator=test_generator)

Epoch 1/2
500/500 [==============================] - 142s 284ms/step - loss: 0.1125 - acc: 0.9533 - val_loss: 0.0549 - val_acc: 0.9688
Epoch 2/2
500/500 [==============================] - 152s 303ms/step - loss: 0.0173 - acc: 0.9937 - val_loss: 0.0068 - val_acc: 1.0000


In [79]:
score

[2.089461636543274, 0.7833333293596904]